# K-means clustering
This module can compute clusters given the number of centroids k as an input, using a variety of seeding methods. It can also automatically select the best k value from a range of suggested k values, using the simplified silhouette method or the elbow method.

## Table of contents

<a href="#setup">0. Setup</a>

<a href="#single_k">1. Clustering for single k value</a>

<a href="#range_k">2. Clustering for a range of k values</a>

<a id="setup"></a>
# 0. Setup

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum Database 5.x on GCP (PM demo machine) - direct external IP access
#%sql postgresql://gpadmin@34.67.65.96:5432/madlib

# Greenplum Database 5.x on GCP - via tunnel
%sql postgresql://gpadmin@localhost:8000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: gpadmin@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.17-dev, git revision: rel/v1.16-54-gec5614f, cmake configuration time: Wed Dec 18 17:08:05 UTC 2019, build type: release, build system: Linux-3.10.0-1062.4.3.el7.x86_64, C compiler: gcc 4.8.5, C++ compiler: g++ 4.8.5"


<a id="single_k"></a>
# Clustering for single k value

# 1. Input data

In [4]:
%%sql
DROP TABLE IF EXISTS km_sample;

CREATE TABLE km_sample(pid int, points double precision[]);

INSERT INTO km_sample VALUES
(1,  '{14.23, 1.71, 2.43, 15.6, 127, 2.8, 3.0600, 0.2800, 2.29, 5.64, 1.04, 3.92, 1065}'),
(2,  '{13.2, 1.78, 2.14, 11.2, 1, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.49, 1050}'),
(3,  '{13.16, 2.36,  2.67, 18.6, 101, 2.8,  3.24, 0.3, 2.81, 5.6799, 1.03, 3.17, 1185}'),
(4,  '{14.37, 1.95, 2.5, 16.8, 113, 3.85, 3.49, 0.24, 2.18, 7.8, 0.86, 3.45, 1480}'),
(5,  '{13.24, 2.59, 2.87, 21, 118, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735}'),
(6,  '{14.2, 1.76, 2.45, 15.2, 112, 3.27, 3.39, 0.34, 1.97, 6.75, 1.05, 2.85, 1450}'),
(7,  '{14.39, 1.87, 2.45, 14.6, 96, 2.5, 2.52, 0.3, 1.98, 5.25, 1.02, 3.58, 1290}'),
(8,  '{14.06, 2.15, 2.61, 17.6, 121, 2.6, 2.51, 0.31, 1.25, 5.05, 1.06, 3.58, 1295}'),
(9,  '{14.83, 1.64, 2.17, 14, 97, 2.8, 2.98, 0.29, 1.98, 5.2, 1.08, 2.85, 1045}'),
(10, '{13.86, 1.35, 2.27, 16, 98, 2.98, 3.15, 0.22, 1.8500, 7.2199, 1.01, 3.55, 1045}');

SELECT * FROM km_sample ORDER BY pid;

Done.
Done.
10 rows affected.
10 rows affected.


pid,points
1,"[14.23, 1.71, 2.43, 15.6, 127.0, 2.8, 3.06, 0.28, 2.29, 5.64, 1.04, 3.92, 1065.0]"
2,"[13.2, 1.78, 2.14, 11.2, 1.0, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.49, 1050.0]"
3,"[13.16, 2.36, 2.67, 18.6, 101.0, 2.8, 3.24, 0.3, 2.81, 5.6799, 1.03, 3.17, 1185.0]"
4,"[14.37, 1.95, 2.5, 16.8, 113.0, 3.85, 3.49, 0.24, 2.18, 7.8, 0.86, 3.45, 1480.0]"
5,"[13.24, 2.59, 2.87, 21.0, 118.0, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735.0]"
6,"[14.2, 1.76, 2.45, 15.2, 112.0, 3.27, 3.39, 0.34, 1.97, 6.75, 1.05, 2.85, 1450.0]"
7,"[14.39, 1.87, 2.45, 14.6, 96.0, 2.5, 2.52, 0.3, 1.98, 5.25, 1.02, 3.58, 1290.0]"
8,"[14.06, 2.15, 2.61, 17.6, 121.0, 2.6, 2.51, 0.31, 1.25, 5.05, 1.06, 3.58, 1295.0]"
9,"[14.83, 1.64, 2.17, 14.0, 97.0, 2.8, 2.98, 0.29, 1.98, 5.2, 1.08, 2.85, 1045.0]"
10,"[13.86, 1.35, 2.27, 16.0, 98.0, 2.98, 3.15, 0.22, 1.85, 7.2199, 1.01, 3.55, 1045.0]"


# 2. Run k-means clustering using kmeans++ with centroid seeding
Use squared Euclidean distance which is a commonly used distance function.

In [5]:
%%sql
DROP TABLE IF EXISTS km_result;

CREATE TABLE km_result AS
SELECT * FROM madlib.kmeanspp( 'km_sample',   -- Table of source data
                               'points',      -- Column containing point co-ordinates 
                               2,             -- Number of centroids to calculate
                               'madlib.squared_dist_norm2',   -- Distance function
                               'madlib.avg',  -- Aggregate function
                               20,            -- Number of iterations
                               0.001          -- Fraction of centroids reassigned to keep iterating 
                             );

SELECT * FROM km_result;

Done.
1 rows affected.
1 rows affected.


centroids,cluster_variance,objective_fn,frac_reassigned,num_iterations
"[[13.872, 1.814, 2.376, 15.56, 88.2, 2.806, 2.928, 0.288, 1.844, 5.35198, 1.044, 3.348, 988.0], [14.036, 2.018, 2.536, 16.56, 108.6, 3.004, 3.03, 0.298, 2.038, 6.10598, 1.004, 3.326, 1340.0]]","[90512.324426408, 60672.638245208]",151184.962672,0.0,2


# 3. Simplified silhouette coefficient
Average for whole data set.  Make sure to use the same distance function as k-means above.

In [6]:
%%sql
SELECT * FROM madlib.simple_silhouette( 'km_sample',          -- Input points table
                                        'points',             -- Column containing points
                                        (SELECT centroids FROM km_result),  -- Centroids
                                        'madlib.squared_dist_norm2'   -- Distance function
                                      );

1 rows affected.


simple_silhouette
0.872087020147


Now calculate simplified silhouette coefficient for each point in the data set:

In [7]:
%%sql
DROP TABLE IF EXISTS km_points_silh;

SELECT * FROM madlib.simple_silhouette_points( 'km_sample',          -- Input points table
                                              'km_points_silh',      -- Output table
                                              'pid',                 -- Point ID column in input table
                                              'points',              -- Points column in input table
                                              'km_result',           -- Centroids table
                                              'centroids',           -- Column in centroids table containing centroids  
                                              'madlib.squared_dist_norm2'   -- Distance function
                                      );

SELECT * FROM km_points_silh ORDER BY pid;

Done.
1 rows affected.
10 rows affected.


pid,centroid_id,neighbor_centroid_id,silh
1,0,1,0.902123603766
2,0,1,0.88017393665
3,1,0,0.382089480836
4,1,0,0.919141622264
5,0,1,0.822664572979
6,1,0,0.943394365443
7,1,0,0.970809939945
8,1,0,0.977109993192
9,0,1,0.961796151461
10,0,1,0.961566534928


# 4. Cluster assignment for each point
Use the closest_column() function to map each point to the cluster that it belongs to. 

In [8]:
%%sql
DROP TABLE IF EXISTS point_cluster_map;

CREATE TABLE point_cluster_map AS 
SELECT data.*, (madlib.closest_column(centroids, points, 'madlib.squared_dist_norm2')).*
FROM km_sample as data, km_result;

ALTER TABLE point_cluster_map RENAME column_id to cluster_id; -- change column name
SELECT * FROM point_cluster_map ORDER BY pid;

Done.
10 rows affected.
Done.
10 rows affected.


pid,points,cluster_id,distance
1,"[14.23, 1.71, 2.43, 15.6, 127.0, 2.8, 3.06, 0.28, 2.29, 5.64, 1.04, 3.92, 1065.0]",0,7435.21009152
2,"[13.2, 1.78, 2.14, 11.2, 1.0, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.49, 1050.0]",0,11468.6944211
3,"[13.16, 2.36, 2.67, 18.6, 101.0, 2.8, 3.24, 0.3, 2.81, 5.6799, 1.03, 3.17, 1185.0]",1,24088.7121562
4,"[14.37, 1.95, 2.5, 16.8, 113.0, 3.85, 3.49, 0.24, 2.18, 7.8, 0.86, 3.45, 1480.0]",1,19623.3917358
5,"[13.24, 2.59, 2.87, 21.0, 118.0, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735.0]",0,64929.1866187
6,"[14.2, 1.76, 2.45, 15.2, 112.0, 3.27, 3.39, 0.34, 1.97, 6.75, 1.05, 2.85, 1450.0]",1,12114.3606538
7,"[14.39, 1.87, 2.45, 14.6, 96.0, 2.5, 2.52, 0.3, 1.98, 5.25, 1.02, 3.58, 1290.0]",1,2664.07117376
8,"[14.06, 2.15, 2.61, 17.6, 121.0, 2.6, 2.51, 0.31, 1.25, 5.05, 1.06, 3.58, 1295.0]",1,2182.10252576
9,"[14.83, 1.64, 2.17, 14.0, 97.0, 2.8, 2.98, 0.29, 1.98, 5.2, 1.08, 2.85, 1045.0]",0,3330.15771392
10,"[13.86, 1.35, 2.27, 16.0, 98.0, 2.98, 3.15, 0.22, 1.85, 7.2199, 1.01, 3.55, 1045.0]",0,3349.07558113


# 5.  Display cluster ID
There are two steps to get the cluster id associated  with the centroid coordinates, if you need it.  First unnest the cluster centroids 2-D array to get a set of 1-D centroid arrays:

In [9]:
%%sql
DROP TABLE IF EXISTS km_centroids_unnest;

-- Run unnest function
CREATE TABLE km_centroids_unnest AS
SELECT (madlib.array_unnest_2d_to_1d(centroids)).*
FROM km_result;

SELECT * FROM km_centroids_unnest ORDER BY 1;

Done.
2 rows affected.
2 rows affected.


unnest_row_id,unnest_result
1,"[13.872, 1.814, 2.376, 15.56, 88.2, 2.806, 2.928, 0.288, 1.844, 5.35198, 1.044, 3.348, 988.0]"
2,"[14.036, 2.018, 2.536, 16.56, 108.6, 3.004, 3.03, 0.298, 2.038, 6.10598, 1.004, 3.326, 1340.0]"


Note that the ID column returned by 'array_unnest_2d_to_1d()' is just a row ID and not the cluster ID assigned by k-means.  The second step to get the cluster_id is:

In [10]:
%%sql
SELECT cent.*,  (madlib.closest_column(centroids, unnest_result, 'madlib.squared_dist_norm2')).column_id as cluster_id
FROM km_centroids_unnest as cent, km_result
ORDER BY cent.unnest_row_id;

2 rows affected.


unnest_row_id,unnest_result,cluster_id
1,"[13.872, 1.814, 2.376, 15.56, 88.2, 2.806, 2.928, 0.288, 1.844, 5.35198, 1.044, 3.348, 988.0]",0
2,"[14.036, 2.018, 2.536, 16.56, 108.6, 3.004, 3.03, 0.298, 2.038, 6.10598, 1.004, 3.326, 1340.0]",1


# 6. Array input
Create the input table:

In [11]:
%%sql
DROP TABLE IF EXISTS km_arrayin CASCADE;

CREATE TABLE km_arrayin(pid int, 
                        p1 float, 
                        p2 float, 
                        p3 float,
                        p4 float, 
                        p5 float, 
                        p6 float,
                        p7 float, 
                        p8 float, 
                        p9 float,
                        p10 float, 
                        p11 float, 
                        p12 float,
                        p13 float);

INSERT INTO km_arrayin VALUES
(1,  14.23, 1.71, 2.43, 15.6, 127, 2.8, 3.0600, 0.2800, 2.29, 5.64, 1.04, 3.92, 1065),
(2,  13.2, 1.78, 2.14, 11.2, 1, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.49, 1050),
(3,  13.16, 2.36,  2.67, 18.6, 101, 2.8,  3.24, 0.3, 2.81, 5.6799, 1.03, 3.17, 1185),
(4,  14.37, 1.95, 2.5, 16.8, 113, 3.85, 3.49, 0.24, 2.18, 7.8, 0.86, 3.45, 1480),
(5,  13.24, 2.59, 2.87, 21, 118, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735),
(6,  14.2, 1.76, 2.45, 15.2, 112, 3.27, 3.39, 0.34, 1.97, 6.75, 1.05, 2.85, 1450),
(7,  14.39, 1.87, 2.45, 14.6, 96, 2.5, 2.52, 0.3, 1.98, 5.25, 1.02, 3.58, 1290),
(8,  14.06, 2.15, 2.61, 17.6, 121, 2.6, 2.51, 0.31, 1.25, 5.05, 1.06, 3.58, 1295),
(9,  14.83, 1.64, 2.17, 14, 97, 2.8, 2.98, 0.29, 1.98, 5.2, 1.08, 2.85, 1045),
(10, 13.86, 1.35, 2.27, 16, 98, 2.98, 3.15, 0.22, 1.8500, 7.2199, 1.01, 3.55, 1045);

SELECT * FROM km_arrayin ORDER BY pid;

Done.
Done.
10 rows affected.
10 rows affected.


pid,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13
1,14.23,1.71,2.43,15.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
2,13.2,1.78,2.14,11.2,1.0,2.65,2.76,0.26,1.28,4.38,1.05,3.49,1050.0
3,13.16,2.36,2.67,18.6,101.0,2.8,3.24,0.3,2.81,5.6799,1.03,3.17,1185.0
4,14.37,1.95,2.5,16.8,113.0,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480.0
5,13.24,2.59,2.87,21.0,118.0,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735.0
6,14.2,1.76,2.45,15.2,112.0,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450.0
7,14.39,1.87,2.45,14.6,96.0,2.5,2.52,0.3,1.98,5.25,1.02,3.58,1290.0
8,14.06,2.15,2.61,17.6,121.0,2.6,2.51,0.31,1.25,5.05,1.06,3.58,1295.0
9,14.83,1.64,2.17,14.0,97.0,2.8,2.98,0.29,1.98,5.2,1.08,2.85,1045.0
10,13.86,1.35,2.27,16.0,98.0,2.98,3.15,0.22,1.85,7.2199,1.01,3.55,1045.0


Now find the cluster assignment for each point using random seeding:

In [12]:
%%sql
DROP TABLE IF EXISTS km_result_array;

-- Run kmeans algorithm
CREATE TABLE km_result_array AS
SELECT * FROM madlib.kmeans_random('km_arrayin',                 -- Table of source data
                                'ARRAY[p1, p2, p3, p4, p5, p6,   -- Points
                                      p7, p8, p9, p10, p11, p12, p13]', 
                                2,                               -- Number of centroids to calculate
                                'madlib.squared_dist_norm2',     -- Distance function
                                'madlib.avg',                    -- Aggregate function
                                20,                              -- Number of iterations
                                0.001);                          -- Fraction of centroids reassigned to keep iterating 

-- Get point assignment
SELECT data.*,  (madlib.closest_column(centroids, 
                                       ARRAY[p1, p2, p3, p4, p5, p6, 
                                      p7, p8, p9, p10, p11, p12, p13], 
                                       'madlib.squared_dist_norm2')).column_id as cluster_id
FROM km_arrayin as data, km_result_array
ORDER BY data.pid;

Done.
1 rows affected.
10 rows affected.


pid,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,cluster_id
1,14.23,1.71,2.43,15.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,1
2,13.2,1.78,2.14,11.2,1.0,2.65,2.76,0.26,1.28,4.38,1.05,3.49,1050.0,1
3,13.16,2.36,2.67,18.6,101.0,2.8,3.24,0.3,2.81,5.6799,1.03,3.17,1185.0,1
4,14.37,1.95,2.5,16.8,113.0,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480.0,0
5,13.24,2.59,2.87,21.0,118.0,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735.0,1
6,14.2,1.76,2.45,15.2,112.0,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450.0,0
7,14.39,1.87,2.45,14.6,96.0,2.5,2.52,0.3,1.98,5.25,1.02,3.58,1290.0,0
8,14.06,2.15,2.61,17.6,121.0,2.6,2.51,0.31,1.25,5.05,1.06,3.58,1295.0,0
9,14.83,1.64,2.17,14.0,97.0,2.8,2.98,0.29,1.98,5.2,1.08,2.85,1045.0,1
10,13.86,1.35,2.27,16.0,98.0,2.98,3.15,0.22,1.85,7.2199,1.01,3.55,1045.0,1


<a id="range_k"></a>
# Clustering for a range of k values

# 1. Auto k selection
Now let's run k-means random for a variety of k values and compare using simple silhouette and elbow methods.

In [13]:
%%sql
DROP TABLE IF EXISTS km_result_auto, km_result_auto_summary;

SELECT madlib.kmeans_random_auto(
    'km_sample',                   -- points table
    'km_result_auto',              -- output table
    'points',                      -- column name in point table
    ARRAY[2,3,4,5,6],              -- k values to try
    'madlib.squared_dist_norm2',   -- distance function
    'madlib.avg',                  -- aggregate function
    20,                            -- max iterations
    0.001,                         -- minimum fraction of centroids reassigned to continue iterating
    'both'                         -- k selection algorithm  (simple silhouette and elbow)
);

SELECT * FROM km_result_auto_summary;

Done.
1 rows affected.
1 rows affected.


k,centroids,cluster_variance,objective_fn,frac_reassigned,num_iterations,silhouette,elbow,selection_algorithm
5,"[[13.24, 2.59, 2.87, 21.0, 118.0, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735.0], [14.3066666666667, 1.56666666666667, 2.29, 15.2, 107.333333333333, 2.86, 3.06333333333333, 0.263333333333333, 2.04, 6.01996666666667, 1.04333333333333, 3.44, 1051.66666666667], [13.2, 1.78, 2.14, 11.2, 1.0, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.49, 1050.0], [13.87, 2.12666666666667, 2.57666666666667, 16.9333333333333, 106.0, 2.63333333333333, 2.75666666666667, 0.303333333333333, 2.01333333333333, 5.32663333333333, 1.03666666666667, 3.44333333333333, 1256.66666666667], [14.285, 1.855, 2.475, 16.0, 112.5, 3.56, 3.44, 0.29, 2.075, 7.275, 0.955, 3.15, 1465.0]]","[0.0, 853.150626673333, 0.0, 8078.22646267333, 452.7633]",9384.14038935,0.0,3,0.953601354123,44857.4267179,silhouette


The best selection above is made by the silhouette algorithm by default.  Note that the elbow method may select a different k value as the best.  To see results for all k values:

In [14]:
%%sql
SELECT * FROM km_result_auto ORDER BY k;

5 rows affected.


k,centroids,cluster_variance,objective_fn,frac_reassigned,num_iterations,silhouette,elbow
2,"[[13.7533333333333, 1.905, 2.425, 16.0666666666667, 90.3333333333333, 2.805, 2.98, 0.29, 2.005, 5.40663333333333, 1.04166666666667, 3.31833333333333, 1020.83333333333], [14.255, 1.9325, 2.5025, 16.05, 110.5, 3.055, 2.9775, 0.2975, 1.845, 6.2125, 0.9975, 3.365, 1378.75]]","[122999.110416013, 30561.74805]",153560.858466,0.0,2,0.86817460894,71506.2870379
3,"[[13.24, 2.59, 2.87, 21.0, 118.0, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735.0], [14.255, 1.9325, 2.5025, 16.05, 110.5, 3.055, 2.9775, 0.2975, 1.845, 6.2125, 0.9975, 3.365, 1378.75], [13.856, 1.768, 2.336, 15.08, 84.8, 2.806, 3.038, 0.27, 2.042, 5.62396, 1.042, 3.396, 1078.0]]","[0.0, 30561.74805, 24007.669589612]",54569.4176396,0.0,4,0.914703320636,38199.4011006
4,"[[13.87, 2.12666666666667, 2.57666666666667, 16.9333333333333, 106.0, 2.63333333333333, 2.75666666666667, 0.303333333333333, 2.01333333333333, 5.32663333333333, 1.03666666666667, 3.44333333333333, 1256.66666666667], [14.2, 1.76, 2.45, 15.2, 112.0, 3.27, 3.39, 0.34, 1.97, 6.75, 1.05, 2.85, 1450.0], [14.37, 1.95, 2.5, 16.8, 113.0, 3.85, 3.49, 0.24, 2.18, 7.8, 0.86, 3.45, 1480.0], [13.872, 1.814, 2.376, 15.56, 88.2, 2.806, 2.928, 0.288, 1.844, 5.35198, 1.044, 3.348, 988.0]]","[8078.22646267333, 0.0, 0.0, 90512.324426408]",98590.5508891,0.0,3,0.894959666277,8221.52797196
5,"[[13.24, 2.59, 2.87, 21.0, 118.0, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735.0], [14.3066666666667, 1.56666666666667, 2.29, 15.2, 107.333333333333, 2.86, 3.06333333333333, 0.263333333333333, 2.04, 6.01996666666667, 1.04333333333333, 3.44, 1051.66666666667], [13.2, 1.78, 2.14, 11.2, 1.0, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.49, 1050.0], [13.87, 2.12666666666667, 2.57666666666667, 16.9333333333333, 106.0, 2.63333333333333, 2.75666666666667, 0.303333333333333, 2.01333333333333, 5.32663333333333, 1.03666666666667, 3.44333333333333, 1256.66666666667], [14.285, 1.855, 2.475, 16.0, 112.5, 3.56, 3.44, 0.29, 2.075, 7.275, 0.955, 3.15, 1465.0]]","[0.0, 853.150626673333, 0.0, 8078.22646267333, 452.7633]",9384.14038935,0.0,3,0.953601354123,44857.4267179
6,"[[13.16, 2.36, 2.67, 18.6, 101.0, 2.8, 3.24, 0.3, 2.81, 5.6799, 1.03, 3.17, 1185.0], [14.37, 1.95, 2.5, 16.8, 113.0, 3.85, 3.49, 0.24, 2.18, 7.8, 0.86, 3.45, 1480.0], [14.04, 1.8225, 2.435, 16.65, 110.0, 2.845, 2.97, 0.295, 1.985, 5.594975, 1.0425, 3.3125, 972.5], [14.2, 1.76, 2.45, 15.2, 112.0, 3.27, 3.39, 0.34, 1.97, 6.75, 1.05, 2.85, 1450.0], [14.225, 2.01, 2.53, 16.1, 108.5, 2.55, 2.515, 0.305, 1.615, 5.15, 1.04, 3.58, 1292.5], [13.2, 1.78, 2.14, 11.2, 1.0, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.49, 1050.0]]","[0.0, 0.0, 76176.4564000075, 0.0, 329.8988, 0.0]",76506.3552,0.0,2,0.772762876324,78164.3126552


# 2. Simplified silhouette for each point for a specific k value

Let's say we want the simplified silhouette coefficient for each point in the data set, for the case where k=3:

In [15]:
%%sql
DROP TABLE IF EXISTS km_points_silh;

SELECT * FROM madlib.simple_silhouette_points( 'km_sample',          -- Input points table
                                              'km_points_silh',     -- Output table
                                              'pid',                -- Point ID column in input table
                                              'points',             -- Points column in input table
                                              (SELECT centroids FROM km_result_auto WHERE k=3), -- centroids array
                                              'madlib.squared_dist_norm2'   -- Distance function
                                      );

SELECT * FROM km_points_silh ORDER BY pid;

Done.
1 rows affected.
10 rows affected.


pid,centroid_id,neighbor_centroid_id,silh
1,2,1,0.980239585977
2,2,0,0.930766555398
3,2,1,0.688472504844
4,1,2,0.93681403243
5,0,2,1.0
6,1,2,0.963483713725
7,1,2,0.820493559534
8,1,2,0.852729167645
9,2,0,0.987157842135
10,2,0,0.986876244672
